In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features
from pandas import ExcelWriter

fig_dir='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Saildrone/'

icefile='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Ice Present.xlsx'
DWfile='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/DiurnalWarming Present.xlsx'

baddir='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/'

data_dir = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission_combined/'

data_dir_out = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission_combined_flags/'



## Read in all the cruise full data

- swap dims from obs to time
- I checked & time = for both trajectories

In [2]:
ds = xr.open_mfdataset(data_dir+'*.nc',combine='nested',concat_dim='trajectory').load()

xlon=ds.lon
time_offset_to_lmt=(xlon/360.)*24.*60
ds['tlmt']=xlon
for i in range(2):
    ds['tlmt'][i,:]= ds.time.data+time_offset_to_lmt[i,:]*np.timedelta64(1,'m')# dt.timedelta(seconds=1)


## read in ice, diurnal warming, and surface coolingpresent file

This file is a combination of the NOAA ice file & own analysis of data based on STD of salinity and SST

In [3]:

badfile=baddir+'Arctic_bad.xlsx'
ds_tem = pd.read_excel(badfile, sheet_name='warm',index_col=1)
ds_warm = ds_tem.to_xarray()
ds_tem = pd.read_excel(badfile, sheet_name='cold',index_col=1)
ds_cool = ds_tem.to_xarray()

tem=ds_warm
for i in range(tem.usv.size):
    tt=pd.to_datetime(str(tem.start_time[i].data),format='%Y-%m-%dT%H:%M:%S')
    tt64=np.datetime64(tt)
    tem.start_time[i]=tt64
    tt=pd.to_datetime(str(tem.stop_time[i].data),format='%Y-%m-%dT%H:%M:%S')
    tt64=np.datetime64(tt)
    tem.stop_time[i]=tt64
ds_warm=tem
tem=ds_cool
for i in range(tem.usv.size):
    tt=pd.to_datetime(str(tem.start_time[i].data),format='%Y-%m-%dT%H:%M:%S')
    tt64=np.datetime64(tt)
    tem.start_time[i]=tt64
    tt=pd.to_datetime(str(tem.stop_time[i].data),format='%Y-%m-%dT%H:%M:%S')
    tt64=np.datetime64(tt)
    tem.stop_time[i]=tt64
ds_cool=tem

tem_mask=np.zeros(ds.TEMP_AIR_MEAN.shape)
tem_mask = xr.DataArray(tem_mask,coords=[ds.trajectory,ds.time],dims=['trajectory','time'],
                                           attrs={'long_name':'Diurnal warming mask 1=warm'})
ds['warm_mask']=tem_mask
for iusv in range(2):
    tem =  ds_warm.sel(usv=int(ds.trajectory[iusv].data))
    ds_tem = ds.isel(trajectory=iusv)
    if tem.usv.size>1:
        for i in range(tem.usv.size):
            cond = (ds_tem.time>=tem.start_time[i]) & (ds_tem.time<=tem.stop_time[i])
            ds_tem.warm_mask[cond]=1
    else:
        cond = (ds_tem.time>=tem.start_time) & (ds_tem.time<=tem.stop_time)
        ds_tem.warm_mask[cond]=1       
    ds['warm_mask'][iusv,:]=ds_tem.warm_mask

tem_mask=np.zeros(ds.TEMP_AIR_MEAN.shape)
tem_mask = xr.DataArray(tem_mask,coords=[ds.trajectory,ds.time],dims=['trajectory','time'],
                                           attrs={'long_name':'surface cooling mask 1=cool'})
ds['cool_mask']=tem_mask
for iusv in range(2):
    tem =  ds_cool.sel(usv=int(ds.trajectory[iusv].data))
    ds_tem = ds.isel(trajectory=iusv)
    if tem.usv.size>1:
        for i in range(tem.usv.size):
            cond = (ds_tem.time>=tem.start_time[i]) & (ds_tem.time<=tem.stop_time[i])
            ds_tem.cool_mask[cond]=1
    else:
        cond = (ds_tem.time>=tem.start_time) & (ds_tem.time<=tem.stop_time)
        ds_tem.cool_mask[cond]=1       
    ds['cool_mask'][iusv,:]=ds_tem.cool_mask


In [5]:
ds.to_netcdf(data_dir_out + 'saildrone-gen_5-arctic_misst_2019-sd10361037-20190514T230000-20191011T183000-1_minutes-v1.1575336154680-withSBE_flags.nc')



In [6]:
ds

<xarray.Dataset>
Dimensions:                         (time: 215731, trajectory: 2)
Coordinates:
  * time                            (time) datetime64[ns] 2019-05-14T23:00:00 ... 2019-10-11T18:30:00
    lat                             (trajectory, time) float64 53.85 ... 53.84
    lon                             (trajectory, time) float64 1.558e+18 ... 1.571e+18
  * trajectory                      (trajectory) object '1036' '1037'
Data variables:
    SOG                             (trajectory, time) float64 1.41 ... 0.005
    SOG_FILTERED_MEAN               (trajectory, time) float64 1.194 ... 0.028
    SOG_FILTERED_STDDEV             (trajectory, time) float64 0.096 ... 0.02
    SOG_FILTERED_MAX                (trajectory, time) float64 1.48 ... 0.095
    SOG_FILTERED_MIN                (trajectory, time) float64 1.041 ... 0.002
    COG                             (trajectory, time) float64 324.0 ... 93.4
    COG_FILTERED_MEAN               (trajectory, time) float64 324.6 ... 83.2
    COG_FILTERED_STDDEV             (trajectory, time) float64 4.1 3.9 ... 90.9
    HDG                             (trajectory, time) float64 317.4 ... 323.7
    HDG_FILTERED_MEAN               (trajectory, time) float64 317.0 ... 323.6
    HDG_FILTERED_STDDEV             (trajectory, time) float64 0.0 0.0 ... 0.0
    ROLL_FILTERED_MEAN              (trajectory, time) float64 2.7 3.2 ... -1.1
    ROLL_FILTERED_STDDEV            (trajectory, time) float64 0.0 0.0 ... 0.0
    ROLL_FILTERED_PEAK              (trajectory, time) float64 2.7 3.2 ... -1.1
    PITCH_FILTERED_MEAN             (trajectory, time) float64 -1.1 ... -0.5
    PITCH_FILTERED_STDDEV           (trajectory, time) float64 0.0 0.0 ... 0.0
    PITCH_FILTERED_PEAK             (trajectory, time) float64 -1.1 ... -0.5
    HDG_WING                        (trajectory, time) float64 242.8 ... 171.7
    WING_HDG_FILTERED_MEAN          (trajectory, time) float64 229.9 ... 171.2
    WING_HDG_FILTERED_STDDEV        (trajectory, time) float64 5.5 3.7 ... 0.5
    WING_ROLL_FILTERED_MEAN         (trajectory, time) float64 0.2 0.8 ... 1.9
    WING_ROLL_FILTERED_STDDEV       (trajectory, time) float64 0.6 0.6 ... 0.1
    WING_ROLL_FILTERED_PEAK         (trajectory, time) float64 nan nan ... nan
    WING_PITCH_FILTERED_MEAN        (trajectory, time) float64 0.4 1.4 ... -1.0
    WING_PITCH_FILTERED_STDDEV      (trajectory, time) float64 0.9 1.2 ... 0.1
    WING_PITCH_FILTERED_PEAK        (trajectory, time) float64 3.3 4.7 ... -1.2
    WING_ANGLE                      (trajectory, time) float64 -74.6 ... -152.0
    UWND_MEAN                       (trajectory, time) float64 4.24 ... -0.23
    UWND_STDDEV                     (trajectory, time) float64 0.52 ... 0.14
    VWND_MEAN                       (trajectory, time) float64 5.59 ... 0.41
    VWND_STDDEV                     (trajectory, time) float64 0.48 ... 0.13
    WWND_MEAN                       (trajectory, time) float64 -0.33 ... -0.08
    WWND_STDDEV                     (trajectory, time) float64 0.32 0.3 ... 0.13
    GUST_WND_MEAN                   (trajectory, time) float64 7.78 ... 0.73
    GUST_WND_STDDEV                 (trajectory, time) float64 0.43 0.48 ... 0.1
    WIND_HEIGHT_MEAN                (trajectory, time) float64 4.998 ... 4.996
    WIND_HEIGHT_STDDEV              (trajectory, time) float64 0.002 ... 0.0
    TEMP_AIR_MEAN                   (trajectory, time) float64 6.32 ... 9.24
    TEMP_AIR_STDDEV                 (trajectory, time) float64 0.01 ... 0.01
    RH_MEAN                         (trajectory, time) float64 68.72 ... 99.84
    RH_STDDEV                       (trajectory, time) float64 0.73 0.85 ... 0.0
    BARO_PRES_MEAN                  (trajectory, time) float64 1.016e+03 ... 1.017e+03
    BARO_PRES_STDDEV                (trajectory, time) float64 0.02 ... 0.03
    PAR_AIR_MEAN                    (trajectory, time) float64 828.0 ... 33.0
    PAR_AIR_STDDEV                  (trajectory, time) float64 5.0 4.0 ... 2.0
  

In [7]:
ds = xr.open_dataset(data_dir_out+'saildrone-gen_5-arctic_misst_2019-sd10361037-20190514T230000-20191011T183000-1_minutes-v1.1575336154680-withSBE_flags.nc')

In [8]:
ds2=ds.where(ds.warm_mask>0,drop=True)
ds2.

<xarray.Dataset>
Dimensions:                         (time: 215731, trajectory: 2)
Coordinates:
  * time                            (time) datetime64[ns] 2019-05-14T23:00:00 ... 2019-10-11T18:30:00
    lat                             (trajectory, time) float64 ...
    lon                             (trajectory, time) float64 ...
  * trajectory                      (trajectory) object '1036' '1037'
Data variables:
    SOG                             (trajectory, time) float64 ...
    SOG_FILTERED_MEAN               (trajectory, time) float64 ...
    SOG_FILTERED_STDDEV             (trajectory, time) float64 ...
    SOG_FILTERED_MAX                (trajectory, time) float64 ...
    SOG_FILTERED_MIN                (trajectory, time) float64 ...
    COG                             (trajectory, time) float64 ...
    COG_FILTERED_MEAN               (trajectory, time) float64 ...
    COG_FILTERED_STDDEV             (trajectory, time) float64 ...
    HDG                             (trajectory, time) float64 ...
    HDG_FILTERED_MEAN               (trajectory, time) float64 ...
    HDG_FILTERED_STDDEV             (trajectory, time) float64 ...
    ROLL_FILTERED_MEAN              (trajectory, time) float64 ...
    ROLL_FILTERED_STDDEV            (trajectory, time) float64 ...
    ROLL_FILTERED_PEAK              (trajectory, time) float64 ...
    PITCH_FILTERED_MEAN             (trajectory, time) float64 ...
    PITCH_FILTERED_STDDEV           (trajectory, time) float64 ...
    PITCH_FILTERED_PEAK             (trajectory, time) float64 ...
    HDG_WING                        (trajectory, time) float64 ...
    WING_HDG_FILTERED_MEAN          (trajectory, time) float64 ...
    WING_HDG_FILTERED_STDDEV        (trajectory, time) float64 ...
    WING_ROLL_FILTERED_MEAN         (trajectory, time) float64 ...
    WING_ROLL_FILTERED_STDDEV       (trajectory, time) float64 ...
    WING_ROLL_FILTERED_PEAK         (trajectory, time) float64 ...
    WING_PITCH_FILTERED_MEAN        (trajectory, time) float64 ...
    WING_PITCH_FILTERED_STDDEV      (trajectory, time) float64 ...
    WING_PITCH_FILTERED_PEAK        (trajectory, time) float64 ...
    WING_ANGLE                      (trajectory, time) float64 ...
    UWND_MEAN                       (trajectory, time) float64 ...
    UWND_STDDEV                     (trajectory, time) float64 ...
    VWND_MEAN                       (trajectory, time) float64 ...
    VWND_STDDEV                     (trajectory, time) float64 ...
    WWND_MEAN                       (trajectory, time) float64 ...
    WWND_STDDEV                     (trajectory, time) float64 ...
    GUST_WND_MEAN                   (trajectory, time) float64 ...
    GUST_WND_STDDEV                 (trajectory, time) float64 ...
    WIND_HEIGHT_MEAN                (trajectory, time) float64 ...
    WIND_HEIGHT_STDDEV              (trajectory, time) float64 ...
    TEMP_AIR_MEAN                   (trajectory, time) float64 ...
    TEMP_AIR_STDDEV                 (trajectory, time) float64 ...
    RH_MEAN                         (trajectory, time) float64 ...
    RH_STDDEV                       (trajectory, time) float64 ...
    BARO_PRES_MEAN                  (trajectory, time) float64 ...
    BARO_PRES_STDDEV                (trajectory, time) float64 ...
    PAR_AIR_MEAN                    (trajectory, time) float64 ...
    PAR_AIR_STDDEV                  (trajectory, time) float64 ...
    TEMP_IR_SKY_HULL_MEAN           (trajectory, time) float64 ...
    TEMP_IR_SKY_HULL_STDDEV         (trajectory, time) float64 ...
    TEMP_IR_SEA_HULL_UNCOMP_MEAN    (trajectory, time) float64 ...
    TEMP_IR_SEA_HULL_UNCOMP_STDDEV  (trajectory, time) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (trajectory, time) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (trajectory, time) float64 ...
    WAVE_DOMINANT_PERIOD            (trajectory, time) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (trajectory, time) float64 ...
    TEMP_SBE37_MEAN            